In [1]:
import pandas as pd
import numpy as np
import time
import importlib.machinery
es = importlib.machinery.SourceFileLoader('extrasense','/home/sac086/extrasensory/extrasense/extrasense.py').load_module()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Recreating Exp1

**Experimental Conditions**: Impersonal, Personal, Hybrid training data

**Experiment Output**: User ID, Method, Run number, Accuracy

**Sampling Method for personal data**: randomly sample the data, no stratification of sample classes

In [2]:
def run_experiment1(user_id):
    #print("Getting personal data...")
    personal_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    timestamps_personal = personal_df.pop('timestamp')
    label_sources_personal = personal_df.pop("label_source")
    
    # train impersonal model
    print("getting impersonal data...")
    impersonal_df = es.get_impersonal_data(leave_users_out=[user_id], data_type="activity", labeled_only=True)
    timestamps_impersonal = impersonal_df.pop('timestamp')
    impersonal_train_labels = impersonal_df.pop("label")
    impersonal_label_sources = impersonal_df.pop('label_source')
    impersonal_ids = impersonal_df.pop("user_id")
    #impersonal_df.dropna()
    
    #impersonal_train_labels = impersonal_train_labels.index[impersonal_df.index]
    
     # standard scale training
    print("Scaling Impersonal...")
    impersonal_scaler = StandardScaler().fit(impersonal_df)
    scaled_impersonal_df = impersonal_scaler.transform(impersonal_df)
    
    impersonal_clf = RandomForestClassifier()
    impersonal_clf.fit(scaled_impersonal_df, impersonal_train_labels)
    
    # setup sampler
    rs = ShuffleSplit(n_splits=5, train_size=100, test_size=100)
    
    run_count = 1
    rows = []
    # run sampler
    for train_ind, test_ind in rs.split(personal_df):
        print("Starting run #%s" % run_count)
        personal_train_df = personal_df.iloc[train_ind]
        personal_train_labels = personal_train_df.pop("label")
        
        val_df = personal_df.iloc[test_ind]
        val_labels = val_df.pop("label")
        
        
        #return personal_train_df, personal_train_labels, impersonal_df, impersonal_train_labels
        hybrid_train_df = pd.concat([impersonal_df, personal_train_df])
        #return impersonal_train_labels, personal_train_labels
        hybrid_train_labels = pd.concat([impersonal_train_labels, personal_train_labels])
        
        # scale 
        personal_scaler = StandardScaler().fit(personal_train_df)
        scaled_personal_df = personal_scaler.transform(personal_train_df)
        
        hybrid_scaler = StandardScaler().fit(hybrid_train_df)
        scaled_hybrid_df = hybrid_scaler.transform(hybrid_train_df)
        
        # build and predict personal model
        personal_clf = RandomForestClassifier()
        personal_clf.fit(scaled_personal_df, personal_train_labels)
        personal_scaled_val_df = personal_scaler.transform(val_df)
        personal_predictions = personal_clf.predict(personal_scaled_val_df)
        
        # build and predict hybrid model
        hybrid_clf = RandomForestClassifier()
        hybrid_clf.fit(scaled_hybrid_df, hybrid_train_labels)
        hybrid_scaled_val_df = hybrid_scaler.transform(val_df)
        hybrid_predictions = hybrid_clf.predict(hybrid_scaled_val_df)
        
        # impersonal predictions
        impersonal_scaled_val_df = impersonal_scaler.transform(val_df)
        impersonal_predictions = impersonal_clf.predict(impersonal_scaled_val_df)
        
        # validate models
        personal_score = accuracy_score(val_labels, personal_predictions)
        hybrid_score = accuracy_score(val_labels, hybrid_predictions)
        impersonal_score = accuracy_score(val_labels, impersonal_predictions)
        
        print("\tRun #%s" % run_count)
        print("\tpersonal : %s" % personal_score)
        print("\thybrid : %s" % hybrid_score)
        print("\timpersonal : %s" % impersonal_score)
        print("\n")
        
        personal_row = {"user_id" : user_id, 
                        "method":"personal", 
                        "run_num" : run_count,
                        "accuracy" : personal_score}
        
        hybrid_row = {"user_id" : user_id, 
                        "method":"hybrid", 
                        "run_num" : run_count,
                        "accuracy" : hybrid_score}
        
        impersonal_row = {"user_id" : user_id, 
                        "method":"impersonal", 
                        "run_num" : run_count,
                        "accuracy" : impersonal_score}
        rows.append(personal_row)
        rows.append(hybrid_row)
        rows.append(impersonal_row)
        
        run_count += 1
    return rows

In [3]:
rows = []

for user_id in es.user_ids:
    print("Getting scores for %s" % user_id)
    start = time.time()
    user_rows = run_experiment1(user_id)
    finish = time.time()
    duration_in_minutes = (finish - start) / 60.
    print("\ttook %s minutes" % (duration_in_minutes))
    rows += user_rows

Getting scores for 098A72A5-E3E5-4F54-A152-BBDA0DF7B694
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.72
	hybrid : 0.66
	impersonal : 0.7


Starting run #2
	Run #2
	personal : 0.75
	hybrid : 0.78
	impersonal : 0.74


Starting run #3
	Run #3
	personal : 0.69
	hybrid : 0.68
	impersonal : 0.61


Starting run #4
	Run #4
	personal : 0.75
	hybrid : 0.75
	impersonal : 0.62


Starting run #5
	Run #5
	personal : 0.8
	hybrid : 0.73
	impersonal : 0.64


	took 2.3718498667081196 minutes
Getting scores for 0A986513-7828-4D53-AA1F-E02D6DF9561B
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.76
	hybrid : 0.7
	impersonal : 0.71


Starting run #2
	Run #2
	personal : 0.84
	hybrid : 0.79
	impersonal : 0.67


Starting run #3
	Run #3
	personal : 0.78
	hybrid : 0.74
	impersonal : 0.61


Starting run #4
	Run #4
	personal : 0.89
	hybrid : 0.73
	impersonal : 0.62


Starting run #5
	Run #5
	personal : 0.81
	hybrid : 0.8
	impersonal 

	Run #5
	personal : 0.78
	hybrid : 0.8
	impersonal : 0.61


	took 2.390131421883901 minutes
Getting scores for 4E98F91F-4654-42EF-B908-A3389443F2E7
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.81
	hybrid : 0.67
	impersonal : 0.59


Starting run #2
	Run #2
	personal : 0.74
	hybrid : 0.63
	impersonal : 0.51


Starting run #3
	Run #3
	personal : 0.78
	hybrid : 0.68
	impersonal : 0.54


Starting run #4
	Run #4
	personal : 0.79
	hybrid : 0.67
	impersonal : 0.5


Starting run #5
	Run #5
	personal : 0.78
	hybrid : 0.69
	impersonal : 0.59


	took 2.407808490594228 minutes
Getting scores for 4FC32141-E888-4BFF-8804-12559A491D8C
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.65
	hybrid : 0.72
	impersonal : 0.62


Starting run #2
	Run #2
	personal : 0.75
	hybrid : 0.76
	impersonal : 0.76


Starting run #3
	Run #3
	personal : 0.76
	hybrid : 0.76
	impersonal : 0.67


Starting run #4
	Run #4
	personal : 0.74
	hybrid : 

	Run #4
	personal : 0.63
	hybrid : 0.7
	impersonal : 0.52


Starting run #5
	Run #5
	personal : 0.67
	hybrid : 0.71
	impersonal : 0.49


	took 2.3500503381093343 minutes
Getting scores for 806289BC-AD52-4CC1-806C-0CDB14D65EB6
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.62
	hybrid : 0.7
	impersonal : 0.56


Starting run #2
	Run #2
	personal : 0.65
	hybrid : 0.66
	impersonal : 0.59


Starting run #3
	Run #3
	personal : 0.62
	hybrid : 0.7
	impersonal : 0.58


Starting run #4
	Run #4
	personal : 0.56
	hybrid : 0.64
	impersonal : 0.44


Starting run #5
	Run #5
	personal : 0.51
	hybrid : 0.5
	impersonal : 0.48


	took 2.373437066872915 minutes
Getting scores for 81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.72
	hybrid : 0.66
	impersonal : 0.65


Starting run #2
	Run #2
	personal : 0.62
	hybrid : 0.55
	impersonal : 0.54


Starting run #3
	Run #3
	personal : 0.65
	hybrid : 0

	Run #3
	personal : 0.75
	hybrid : 0.68
	impersonal : 0.69


Starting run #4
	Run #4
	personal : 0.69
	hybrid : 0.66
	impersonal : 0.72


Starting run #5
	Run #5
	personal : 0.66
	hybrid : 0.67
	impersonal : 0.62


	took 2.367266472180684 minutes
Getting scores for BEF6C611-50DA-4971-A040-87FB979F3FC1
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.76
	hybrid : 0.68
	impersonal : 0.52


Starting run #2
	Run #2
	personal : 0.75
	hybrid : 0.73
	impersonal : 0.57


Starting run #3
	Run #3
	personal : 0.78
	hybrid : 0.67
	impersonal : 0.61


Starting run #4
	Run #4
	personal : 0.76
	hybrid : 0.68
	impersonal : 0.56


Starting run #5
	Run #5
	personal : 0.76
	hybrid : 0.65
	impersonal : 0.5


	took 2.3926584402720135 minutes
Getting scores for C48CE857-A0DD-4DDB-BEA5-3A25449B2153
getting impersonal data...
Scaling Impersonal...
Starting run #1
	Run #1
	personal : 0.64
	hybrid : 0.65
	impersonal : 0.58


Starting run #2
	Run #2
	personal : 0.61
	hybrid 

In [4]:
scores_df = pd.DataFrame(rows)

In [6]:
user_ids = scores_df['user_id'].unique()

personal_mean_scores = []
hybrid_mean_scores = []
impersonal_mean_scores = []

for user_id in user_ids:
    user_personal_mean = scores_df[(scores_df['user_id'] == user_id) &\
          (scores_df['method'] == 'personal')]['accuracy'].mean()
    user_hybrid_mean = scores_df[(scores_df['user_id'] == user_id) &\
          (scores_df['method'] == 'hybrid')]['accuracy'].mean()
    user_impersonal_mean = scores_df[(scores_df['user_id'] == user_id) &\
          (scores_df['method'] == 'impersonal')]['accuracy'].mean()
    personal_mean_scores.append(user_personal_mean)
    hybrid_mean_scores.append(user_hybrid_mean)
    impersonal_mean_scores.append(user_impersonal_mean)
    
print("Personal: M=%.3f, SD=%.3f" % (np.mean(personal_mean_scores), np.std(personal_mean_scores)))
print("Hybrid: M=%.3f, SD=%.3f" % (np.mean(hybrid_mean_scores), np.std(hybrid_mean_scores)))
print("Impersonal: M=%.3f, SD=%.3f" % (np.mean(impersonal_mean_scores), np.std(impersonal_mean_scores)))



Personal: M=0.715, SD=0.083
Hybrid: M=0.662, SD=0.065
Impersonal: M=0.575, SD=0.089


# Trying Stratified Shuffle with parallelization

In [2]:
import ipyparallel as ipp

In [3]:
c = ipp.Client()

In [4]:
dview = c[:]

In [5]:
dview.block=True

In [6]:
dview.scatter('user_ids', es.user_ids)

In [7]:
import_string = '''import pandas as pd
import numpy as np
import time
import importlib.machinery
es = importlib.machinery.SourceFileLoader('extrasense','/home/sac086/extrasensory/extrasense/extrasense.py').load_module()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler'''

In [8]:
asr = dview.execute(import_string)

In [21]:
%%px
def run_experiment1(user_id, training_size=100, learning_algo=es.weka_RF, stratified=False):
    #print("Getting personal data...")
    personal_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    timestamps_personal = personal_df.pop('timestamp')
    label_sources_personal = personal_df.pop("label_source")
    
    # train impersonal model
    print("getting impersonal data...")
    impersonal_df = es.get_impersonal_data(leave_users_out=[user_id], data_type="activity", labeled_only=True)
    timestamps_impersonal = impersonal_df.pop('timestamp')
    impersonal_train_labels = impersonal_df.pop("label")
    impersonal_label_sources = impersonal_df.pop('label_source')
    impersonal_ids = impersonal_df.pop("user_id")
    #impersonal_df.dropna()
    
    #impersonal_train_labels = impersonal_train_labels.index[impersonal_df.index]
    
     # standard scale training
    print("Scaling Impersonal...")
    impersonal_scaler = StandardScaler().fit(impersonal_df)
    scaled_impersonal_df = impersonal_scaler.transform(impersonal_df)
    
    impersonal_clf = learning_algo()
    impersonal_clf.fit(scaled_impersonal_df, impersonal_train_labels)
    
    # setup sampler
    rs = ShuffleSplit(n_splits=5, train_size=training_size, test_size=100)
    
    run_count = 1
    rows = []
    # run sampler
    for train_ind, test_ind in rs.split(personal_df):
        print("Starting run #%s" % run_count)
        personal_train_df = personal_df.iloc[train_ind]
        personal_train_labels = personal_train_df.pop("label")
        
        val_df = personal_df.iloc[test_ind]
        val_labels = val_df.pop("label")
        
        
        #return personal_train_df, personal_train_labels, impersonal_df, impersonal_train_labels
        hybrid_train_df = pd.concat([impersonal_df, personal_train_df])
        #return impersonal_train_labels, personal_train_labels
        hybrid_train_labels = pd.concat([impersonal_train_labels, personal_train_labels])
        
        # scale 
        personal_scaler = StandardScaler().fit(personal_train_df)
        scaled_personal_df = personal_scaler.transform(personal_train_df)
        
        hybrid_scaler = StandardScaler().fit(hybrid_train_df)
        scaled_hybrid_df = hybrid_scaler.transform(hybrid_train_df)
        
        # build and predict personal model
        personal_clf = learning_algo()
        personal_clf.fit(scaled_personal_df, personal_train_labels)
        personal_scaled_val_df = personal_scaler.transform(val_df)
        personal_predictions = personal_clf.predict(personal_scaled_val_df)
        
        # build and predict hybrid model
        hybrid_clf = learning_algo()
        hybrid_clf.fit(scaled_hybrid_df, hybrid_train_labels)
        hybrid_scaled_val_df = hybrid_scaler.transform(val_df)
        hybrid_predictions = hybrid_clf.predict(hybrid_scaled_val_df)
        
        # impersonal predictions
        impersonal_scaled_val_df = impersonal_scaler.transform(val_df)
        impersonal_predictions = impersonal_clf.predict(impersonal_scaled_val_df)
        
        # validate models
        personal_score = accuracy_score(val_labels, personal_predictions)
        hybrid_score = accuracy_score(val_labels, hybrid_predictions)
        impersonal_score = accuracy_score(val_labels, impersonal_predictions)
        
        print("\tRun #%s" % run_count)
        print("\tpersonal : %s" % personal_score)
        print("\thybrid : %s" % hybrid_score)
        print("\timpersonal : %s" % impersonal_score)
        print("\n")
        
        personal_row = {"user_id" : user_id, 
                        "method":"personal", 
                        "run_num" : run_count,
                        "training_size" : training_size,
                        "accuracy" : personal_score}
        
        hybrid_row = {"user_id" : user_id, 
                        "method":"hybrid", 
                        "run_num" : run_count,
                        "training_size" : training_size,
                        "accuracy" : hybrid_score}
        
        impersonal_row = {"user_id" : user_id, 
                        "method":"impersonal", 
                        "run_num" : run_count,
                        "training_size" : training_size,
                        "accuracy" : impersonal_score}
        rows.append(personal_row)
        rows.append(hybrid_row)
        rows.append(impersonal_row)
        
        run_count += 1
    return rows

In [ ]:
# without class stratification

command2 = '''
rows = []
errors = []
training_sizes = [5,10,20,30,40,50,60]
for user_id in user_ids:
    for ts in training_sizes:
        print("Getting scores for %s" % user_id)
        start = time.time()
        try:
            user_rows = run_experiment1(user_id, training_size=ts)
        except ValueError as ve:
            errors.append(ve)
            continue
        finish = time.time()
        duration_in_minutes = (finish - start) / 60.
        print("\ttook %s minutes" % (duration_in_minutes))
        rows += user_rows
'''
asr = dview.execute(command2)
rows = dview.gather('rows')
scores_df = pd.DataFrame(rows)
scores_df.to_pickle('./scores_df.pickle')

In [ ]:
# with class stratification

command2 = '''
rows = []
errors = []
training_sizes = [5,10,20,30,40,50,60]
for user_id in user_ids:
    for ts in training_sizes:
        print("Getting scores for %s" % user_id)
        start = time.time()
        try:
            user_rows = run_experiment1(user_id, training_size=ts, stratified=True)
        except ValueError as ve:
            errors.append(ve)
            continue
        finish = time.time()
        duration_in_minutes = (finish - start) / 60.
        print("\ttook %s minutes" % (duration_in_minutes))
        rows += user_rows
'''
asr = dview.execute(command2)
rows = dview.gather('rows')
scores_df = pd.DataFrame(rows)
scores_df.to_pickle('./scores_df_stratified.pickle')

In [19]:
scores_df.head()

,accuracy,method,run_num,training_size,user_id
0,0.37,personal,1,5,098A72A5-E3E5-4F54-A152-BBDA0DF7B694
1,0.60,hybrid,1,5,098A72A5-E3E5-4F54-A152-BBDA0DF7B694
2,0.55,impersonal,1,5,098A72A5-E3E5-4F54-A152-BBDA0DF7B694
3,0.33,personal,2,5,098A72A5-E3E5-4F54-A152-BBDA0DF7B694
4,0.60,hybrid,2,5,098A72A5-E3E5-4F54-A152-BBDA0DF7B694


# Visualizing the results

In [42]:
# Load results from file
stratified_scores_df = pd.read_pickle("./scores_df_stratified.pickle")
not_stratified_scores_df = pd.read_pickle("./scores_df.pickle")

## Results without Stratification

### impersonal mean scores will only include the validation set from the round where personal training data was 5 in order to have a consistent amount of trials as other methods

In [43]:
impersonal_mean_scores = []

for user_id in user_ids:
    user_impersonal_mean = not_stratified_scores_df[(not_stratified_scores_df['user_id'] == user_id) &\
                                     (not_stratified_scores_df['method'] == 'impersonal') &\
                                     (not_stratified_scores_df['training_size'] == 5)]['accuracy'].mean()
    impersonal_mean_scores.append(user_impersonal_mean)

In [44]:
# Results without stratification
print("Impersonal: M=%.3f, SD=%.3f\n" % (np.mean(impersonal_mean_scores), np.std(impersonal_mean_scores)))

user_ids = not_stratified_scores_df['user_id'].unique()
training_sizes = [5,10,20,30,40]

all_personal_scores = []
all_personal_sizes = []

all_hybrid_scores = []
all_hybrid_sizes = []
for ts in training_sizes:
    personal_mean_scores = []
    hybrid_mean_scores = []
    
    for user_id in user_ids:
        user_personal_mean = not_stratified_scores_df[(not_stratified_scores_df['user_id'] == user_id) &\
              (not_stratified_scores_df['method'] == 'personal') &\
              (not_stratified_scores_df['training_size'] == ts)]['accuracy'].mean()
        user_hybrid_mean = not_stratified_scores_df[(not_stratified_scores_df['user_id'] == user_id) &\
              (not_stratified_scores_df['method'] == 'hybrid') &\
              (not_stratified_scores_df['training_size'] == ts)]['accuracy'].mean()
        personal_mean_scores.append(user_personal_mean)
        hybrid_mean_scores.append(user_hybrid_mean)
    
    print("Training Size : %s" % ts)
    print("\tPersonal: M=%.3f, SD=%.3f" % (np.mean(personal_mean_scores), np.std(personal_mean_scores)))
    print("\tHybrid: M=%.3f, SD=%.3f" % (np.mean(hybrid_mean_scores), np.std(hybrid_mean_scores)))
    
    all_personal_scores +=  personal_mean_scores
    all_personal_sizes += [ts] * len(personal_mean_scores)
    
    all_hybrid_scores += hybrid_mean_scores
    all_hybrid_sizes += [ts] * len(hybrid_mean_scores)


Impersonal: M=0.604, SD=0.098

Training Size : 5
	Personal: M=0.557, SD=0.106
	Hybrid: M=0.617, SD=0.092
Training Size : 10
	Personal: M=0.601, SD=0.092
	Hybrid: M=0.629, SD=0.081
Training Size : 20
	Personal: M=0.654, SD=0.090
	Hybrid: M=0.645, SD=0.086
Training Size : 30
	Personal: M=0.683, SD=0.084
	Hybrid: M=0.656, SD=0.078
Training Size : 40
	Personal: M=0.699, SD=0.077
	Hybrid: M=0.676, SD=0.077


In [32]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [46]:
impersonal_trace = go.Box(y=impersonal_mean_scores,
                          x=0,
                          boxpoints='all',
                          jitter=0.8,
                          pointpos=-1,
                          name="Impersonal")


personal_trace = go.Box(y=all_personal_scores,
                       x=all_personal_sizes,
                        boxpoints='all',
                          jitter=0.8,
                          pointpos=-1,
                       name="personal")

hybrid_trace = go.Box(y=all_hybrid_scores,
                     x=all_hybrid_sizes,
                      boxpoints='all',
                          jitter=0.8,
                          pointpos=-1,
                     name="hybrid")

data = [impersonal_trace, personal_trace, hybrid_trace]
layout = go.Layout(yaxis=dict(title='Accuracy', range=[0,1]),
                   xaxis=dict(title='Amount of personal training data'),
                   boxmode='group',
                   title="Accuracy Scores By Method Without Class Stratification"
                  )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Results with Stratification

In [47]:
# Results without stratification
print("Impersonal: M=%.3f, SD=%.3f\n" % (np.mean(impersonal_mean_scores), np.std(impersonal_mean_scores)))

user_ids = stratified_scores_df['user_id'].unique()
training_sizes = [5,10,20,30,40]

all_personal_scores = []
all_personal_sizes = []

all_hybrid_scores = []
all_hybrid_sizes = []
for ts in training_sizes:
    personal_mean_scores = []
    hybrid_mean_scores = []
    
    for user_id in user_ids:
        user_personal_mean = stratified_scores_df[(stratified_scores_df['user_id'] == user_id) &\
              (stratified_scores_df['method'] == 'personal') &\
              (stratified_scores_df['training_size'] == ts)]['accuracy'].mean()
        user_hybrid_mean = stratified_scores_df[(stratified_scores_df['user_id'] == user_id) &\
              (stratified_scores_df['method'] == 'hybrid') &\
              (stratified_scores_df['training_size'] == ts)]['accuracy'].mean()
        personal_mean_scores.append(user_personal_mean)
        hybrid_mean_scores.append(user_hybrid_mean)
    
    print("Training Size : %s" % ts)
    print("\tPersonal: M=%.3f, SD=%.3f" % (np.mean(personal_mean_scores), np.std(personal_mean_scores)))
    print("\tHybrid: M=%.3f, SD=%.3f" % (np.mean(hybrid_mean_scores), np.std(hybrid_mean_scores)))
    
    all_personal_scores +=  personal_mean_scores
    all_personal_sizes += [ts] * len(personal_mean_scores)
    
    all_hybrid_scores += hybrid_mean_scores
    all_hybrid_sizes += [ts] * len(hybrid_mean_scores)


Impersonal: M=0.604, SD=0.098

Training Size : 5
	Personal: M=0.569, SD=0.108
	Hybrid: M=0.614, SD=0.088
Training Size : 10
	Personal: M=0.604, SD=0.098
	Hybrid: M=0.635, SD=0.085
Training Size : 20
	Personal: M=0.663, SD=0.087
	Hybrid: M=0.655, SD=0.083
Training Size : 30
	Personal: M=0.674, SD=0.086
	Hybrid: M=0.658, SD=0.076
Training Size : 40
	Personal: M=0.695, SD=0.085
	Hybrid: M=0.672, SD=0.076
